# Bloque 2 · Conectando RAG con la nube

En este bloque daremos el salto de la simulación local a un servicio en la nube. Presentaremos conceptos básicos de computación en la nube, APIs y Amazon Bedrock, para luego analizar cómo luce la respuesta del endpoint `Retrieve` cuando contamos con una knowledge base ya configurada.

## De la simulación a la nube

- En el **Bloque 0** visualizamos la arquitectura completa y los objetivos del proyecto.
- En el **Bloque 1** exploramos la recuperación basada en similitud con una simulación local.

Ahora nos enfocamos en cómo un servicio administrado nos ayuda a escalar esa lógica, manteniendo el hilo conductor: recuperar fragmentos relevantes para enriquecer respuestas generadas.

## Conceptos clave: nube y APIs

- **Computación en la nube**: consumo de recursos (cómputo, almacenamiento, modelos) como servicios bajo demanda, sin gestionar infraestructura física.
- **API (Application Programming Interface)**: conjunto de reglas para que dos sistemas se comuniquen. Usualmente exponemos recursos vía URLs (endpoints) que aceptan solicitudes con parámetros y devuelven respuestas estructuradas (JSON).
- **Endpoint**: dirección específica de una API que realiza una acción. Por ejemplo, `Retrieve` es un endpoint que consulta una base de conocimiento con una pregunta.

## ¿Qué ofrece Amazon Bedrock?

Amazon Bedrock es un servicio administrado que permite acceder a modelos fundacionales (propios y de terceros) mediante una API unificada. Para nuestro flujo RAG, destacamos:
- **Modelos de embeddings** para convertir texto en vectores.
- **Modelos generativos** para elaborar respuestas.
- **Knowledge bases** que almacenan representaciones vectoriales y resuelven la recuperación de forma administrada.
- **Integración con otras herramientas AWS** (S3, IAM, CloudWatch) que facilitan la operación en producción.

## Knowledge Bases en Bedrock

Una *knowledge base* es un repositorio vectorial que Bedrock administra por nosotros. Sus componentes principales son:
- **Fuentes de datos**: conjuntos de documentos almacenados, por ejemplo, en Amazon S3. Podemos proveer manuales, políticas, FAQs, etc.
- **Pipelines de ingesta**: procesos que limpian, fragmentan y generan embeddings de cada documento.
- **Catálogo de metadatos**: guarda referencias al origen de cada fragmento (ruta de archivo, sección, versión) para poder citar las fuentes en las respuestas.

Asumimos que la organización ya configuró la knowledge base y ejecutó al menos una ingesta para tener información disponible.

## Endpoint `Retrieve`

- Recibe una **pregunta** o consulta en texto.
- Calcula embeddings con el modelo elegido.
- Busca los fragmentos más relevantes en la knowledge base.
- Devuelve un listado ordenado de resultados con puntuaciones, contenido y metadatos.

Esta respuesta luego se utiliza para generar una contestación final (por ejemplo, desde Chainlit) o simplemente para mostrar evidencias al usuario.

## Actividad práctica · Paseo por la respuesta de `Retrieve`

Simularemos la respuesta JSON que entrega Bedrock cuando invocamos el endpoint `Retrieve`. Analizaremos cada campo para entender qué información recibimos y cómo podemos utilizarla en nuestro chatbot.

In [ ]:
import textwrap
from typing import Dict, Any

retrieve_response: Dict[str, Any] = {
    "knowledgeBaseId": "kb-1234567890",
    "modelArn": "arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-embed-text-v2",
    "query": "¿Cómo prepara Bedrock una knowledge base para responder preguntas?",
    "retrievalResults": [
        {
            "score": 0.83,
            "content": [
                {
                    "text": (
                        "Durante la ingesta, Bedrock normaliza el texto, lo divide en chunks y genera "
                        "embeddings con el modelo seleccionado. Los vectores se almacenan en un índice "
                        "optimizado para búsquedas de similitud."
                    )
                }
            ],
            "metadata": {
                "title": "Guía de operación RAG",
                "source": "s3://org-knowledge/rag/guia-operacion.pdf",
                "chunkId": "guia-operacion.pdf:12",
                "authors": ["Equipo Datos"],
            },
            "location": {
                "type": "S3",
                "s3Location": {
                    "uri": "s3://org-knowledge/rag/guia-operacion.pdf",
                    "region": "us-east-1",
                },
            },
        },
        {
            "score": 0.74,
            "content": [
                {
                    "text": (
                        "Una knowledge base puede conectarse a múltiples fuentes. Cada ejecución "
                        "de ingesta detecta cambios y actualiza embeddings para mantener la información al día."
                    )
                }
            ],
            "metadata": {
                "title": "Procedimiento de ingesta",
                "source": "s3://org-knowledge/rag/procedimiento-ingesta.md",
                "chunkId": "procedimiento-ingesta.md:05",
            },
            "location": {
                "type": "S3",
                "s3Location": {
                    "uri": "s3://org-knowledge/rag/procedimiento-ingesta.md",
                    "region": "us-east-1",
                },
            },
        },
        {
            "score": 0.65,
            "content": [
                {
                    "text": (
                        "Las respuestas generadas pueden incluir citas apuntando a los fragmentos recuperados. "
                        "Esto aumenta la transparencia para el usuario final."
                    )
                }
            ],
            "metadata": {
                "title": "Diseño del chatbot",
                "source": "s3://org-knowledge/rag/diseño-chatbot.docx",
                "chunkId": "diseño-chatbot.docx:07",
            },
            "location": {
                "type": "S3",
                "s3Location": {
                    "uri": "s3://org-knowledge/rag/diseño-chatbot.docx",
                    "region": "us-east-1",
                },
            },
        },
    ],
    "nextToken": None,
}

print("Consulta simulada:")
print(retrieve_response["query"])
print("\nTotal de resultados:", len(retrieve_response["retrievalResults"]))


Observa que la respuesta incluye:
- El identificador de la knowledge base utilizada.
- El ARN del modelo de embeddings.
- La pregunta original (`query`).
- Un arreglo de `retrievalResults` con puntajes (`score`), contenido y metadatos.
- Información de la ubicación del documento para citar la fuente.

Exploraremos cada resultado para entender qué datos podemos mostrar al usuario o reutilizar en procesos posteriores.

In [ ]:
def mostrar_resultados(respuesta: Dict[str, Any], max_ancho: int = 96) -> None:
    for idx, resultado in enumerate(respuesta.get("retrievalResults", []), start=1):
        score = round(resultado.get("score", 0.0) * 100, 2)
        metadatos = resultado.get("metadata", {})
        texto = resultado.get("content", [{}])[0].get("text", "")

        print(f"Resultado #{idx}")
        print(f"Similitud: {score}%")
        print(f"Título: {metadatos.get('title', 'Sin título')}")
        print(f"Fuente: {metadatos.get('source', 'Sin referencia')}")
        print("Fragmento:")
        for linea in textwrap.wrap(texto, width=max_ancho):
            print(f"  {linea}")
        print("-" * max_ancho)

mostrar_resultados(retrieve_response)


### Experimenta

- Ajusta el contenido de `retrieve_response` para simular resultados de otra pregunta.
- Agrega metadatos propios (por ejemplo, etiquetas de seguridad o fechas).
- Observa cómo podrías transformar esta información en tarjetas dentro de una interfaz Chainlit o en citas dentro de una respuesta generada.

## Próximos pasos

En el siguiente bloque conectaremos esta recuperación con un modelo generativo en Bedrock y construiremos la interacción en Chainlit, reutilizando los fragmentos y metadatos que acabamos de analizar.